In [13]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_decision_regions

sns.set_style('darkgrid')
%matplotlib inline

In [14]:
# "DisbursementDate" for duplication
sba = pd.read_csv('../data/SBAnational.csv')
df = sba.copy()
df.City = df.City.str.upper()
# drop duplication
df.drop_duplicates(subset=None,keep='first',inplace=True)

C:\Users\wyc\anaconda3\envs\eods-s22\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
df = df[['NAICS','ApprovalFY','Term','NewExist','FranchiseCode','UrbanRural','RevLineCr','MIS_Status','GrAppv']]
# drop na (relatively NOT large data loss)
df.dropna(inplace=True)
df.isnull().sum()

NAICS            0
ApprovalFY       0
Term             0
NewExist         0
FranchiseCode    0
UrbanRural       0
RevLineCr        0
MIS_Status       0
GrAppv           0
dtype: int64

In [16]:
df.NAICS.unique()

array([451120, 722410, 621210, ..., 315280, 922140, 221121], dtype=int64)

In [17]:
# keep first 2 digits of NAICS
df.NAICS = pd.to_numeric(df.NAICS.astype(str).str[:2])
df = df[df.NAICS != 0]
# RevLineCr = 0, 1
df = df[(df.RevLineCr == 'N') | (df.RevLineCr == 'Y')]
df.RevLineCr.replace(['N', 'Y'],[0, 1], inplace=True)
df.RevLineCr = pd.to_numeric(df.RevLineCr)
# Franchise Code = 0, 1
df.FranchiseCode[df.FranchiseCode <= 1] = 0
df.FranchiseCode[df.FranchiseCode > 1] = 1
df = df.rename(columns={"FranchiseCode":"HasFranchise"})
# New Exist = 0, 1 (Delet NewExist = 0.0)
df.NewExist = df.NewExist.astype(int)
df = df[(df.NewExist == 1) | (df.NewExist == 2)]
df.NewExist[df.NewExist == 1] = 0
df.NewExist[df.NewExist == 2] = 1
# Default 
df.MIS_Status.replace(['P I F', 'CHGOFF'],[0, 1], inplace=True)
df = df.rename(columns={"MIS_Status":"Default"})
df.Default = pd.to_numeric(df.Default)
# Industry
df['Industry'] = df['NAICS'].map({
    11: 'Ag/For/Fish/Hunt',
    21: 'Min/Quar/Oil_Gas_ext',
    22: 'Utilities',
    23: 'Construction',
    31: 'Manufacturing',
    32: 'Manufacturing',
    33: 'Manufacturing',
    42: 'Wholesale_trade',
    44: 'Retail_trade',
    45: 'Retail_trade',
    48: 'Trans/Ware',
    49: 'Trans/Ware',
    51: 'Information',
    52: 'Finance/Insurance',
    53: 'RE/Rental/Lease',
    54: 'Prof/Science/Tech',
    55: 'Mgmt_comp',
    56: 'Admin_sup/Waste_Mgmt_Rem',
    61: 'Educational',
    62: 'Healthcare/Social_assist',
    71: 'Arts/Entertain/Rec',
    72: 'Accom/Food_serv',
    81: 'Other_no_pub',
    92: 'Public_Admin'
})
# $, A, ...
df.GrAppv = df.GrAppv.apply(lambda x: x.strip('$'))
df.GrAppv = df.GrAppv.apply(lambda x : x.replace(',',''))
df.GrAppv = pd.to_numeric(df.GrAppv)
df.ApprovalFY[df.ApprovalFY == "1976A"] = df.ApprovalFY[df.ApprovalFY == "1976A"].apply(lambda x: x.strip('A'))
# Change type
df = df.astype({"GrAppv":'int', "ApprovalFY":'int'}) 
# UrbanRural
df = df[df.UrbanRural != 0]
df.UrbanRural[df.UrbanRural == 2] = 0
df.shape

C:\Users\wyc\AppData\Local\Temp\ipykernel_28468\3157474579.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.FranchiseCode[df.FranchiseCode <= 1] = 0
C:\Users\wyc\AppData\Local\Temp\ipykernel_28468\3157474579.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.FranchiseCode[df.FranchiseCode > 1] = 1
C:\Users\wyc\AppData\Local\Temp\ipykernel_28468\3157474579.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.NewExist[df.Ne

(353850, 10)

In [18]:
pd.value_counts(df.NewExist)

0    258351
1     95499
Name: NewExist, dtype: int64

In [19]:
df.to_csv('../data/SBA_cleaned_data_moredeleted.csv')